## FAIR DATA

Los datos provienen del Índice de Uso Justo (FAIR USE) de la Oficina de Derechos de Autor de EE. UU.

**El uso legítimo es un aspecto vital y de larga data de la ley de derechos de autor estadounidense. El objetivo del Índice es hacer que los principios y la aplicación del uso legítimo sean más accesibles y comprensibles para el público mediante la presentación de una base de datos con capacidad de búsqueda de opiniones judiciales, incluso por categoría y tipo de uso (por ejemplo, música, Internet/digitalización, parodia).**

Hay dos conjuntos de datos para los cuales las filas se alinean, pero es posible que los valores no se alineen con precisión para una unión limpia.

Fuente de datos link (https://github.com/rfordatascience/tidytuesday/blob/master/data/2023/2023-08-29/readme.md)

In [1]:
# Importamos librerias
import pandas as pd
pd.set_option('display.max_colwidth', None)
import mysql.connector
from sqlalchemy import create_engine
import pymysql

### Lectura de archivos csv

In [2]:
fair_use_cases_df = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2023/2023-08-29/fair_use_cases.csv')
fair_use_findings_df = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2023/2023-08-29/fair_use_findings.csv')

In [3]:
print(fair_use_cases_df.info())
fair_use_cases_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251 entries, 0 to 250
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   case            251 non-null    object
 1   year            251 non-null    int64 
 2   court           251 non-null    object
 3   jurisdiction    251 non-null    object
 4   categories      251 non-null    object
 5   outcome         251 non-null    object
 6   fair_use_found  251 non-null    bool  
dtypes: bool(1), int64(1), object(5)
memory usage: 12.1+ KB
None


,case,year,court,jurisdiction,categories,outcome,fair_use_found
0,"De Fontbrune v. Wofsy, 39 F.4th 1214 (9th Cir. 2022)",2022,9th Circuit,9th Circuit,Education/Scholarship/Research; Photograph,Fair use not found,False
1,"Sedlik v. Von Drachenberg, No. CV 21-1102 (C.D. Cal. May 31, 2022)",2022,C.D. Cal.,9th Circuit,Painting/Drawing/Graphic; Photograph,Preliminary finding; Fair use not found,False
2,"Sketchworks Indus. Strength Comedy, Inc. v. Jacobs, No. 19-CV-7470-LTS-VF (S.D.N.Y. May 12, 2022)",2022,S.D.N.Y.,2nd Circuit,Film/Audiovisual; Music; Parody/Satire; Review/Commentary,Fair use found,True
3,"Am. Soc'y for Testing & Materials v. Public.Resource.Org, Inc., No. 13-cv-1215 (D.D.C. Mar. 31, 2022)",2022,D.D.C.,District of Columbia Circuit,Education/Scholarship/Research; Textual Work; Used in government proceeding,Mixed Result,False
4,"Yang v. Mic Network Inc., Nos. 20-4097-cv, 20-4201-cv (2d Cir. Mar. 29, 2022)",2022,2d Circuit,2nd Circuit,News reporting; Photography,Fair use found,True


In [4]:
fair_use_cases_df.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
case,251,251,"De Fontbrune v. Wofsy, 39 F.4th 1214 (9th Cir. 2022)",1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
year,251.0,NaN,NaN,NaN,2003.697211,18.226902,1841.0,1993.5,2009.0,2017.0,2022.0
court,251,53,S.D.N.Y.,47,NaN,NaN,NaN,NaN,NaN,NaN,NaN
jurisdiction,251,14,2nd Circuit,98,NaN,NaN,NaN,NaN,NaN,NaN,NaN
categories,251,155,Education/Scholarship/Research; Textual work,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN
outcome,251,16,Fair use not found,102,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fair_use_found,251,2,False,150,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
print(fair_use_findings_df.info())
fair_use_findings_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251 entries, 0 to 250
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        251 non-null    object
 1   case_number  251 non-null    object
 2   year         251 non-null    object
 3   court        251 non-null    object
 4   key_facts    251 non-null    object
 5   issue        251 non-null    object
 6   holding      251 non-null    object
 7   tags         251 non-null    object
 8   outcome      251 non-null    object
dtypes: object(9)
memory usage: 17.8+ KB
None


title  \
0                                           De Fontbrune v. Wofsy   
1                                       Sedlik v. Von Drachenberg   
2              Sketchworks Indus. Strength Comedy, Inc. v. Jacobs   
3  Am. Soc'y for Testing & Materials v. Public.Resource.Org, Inc.   
4                                        Yang v. Mic Network Inc.   

                                                                               case_number  \
0                                                            39 F.4th 1214 (9th Cir. 2022)   
1                                 No. CV 21-1102, 2022 WL 2784818 (C.D. Cal. May 31, 2022)   
2               No. 19-CV-7470-LTS-VF, 2022 U.S. Dist. LEXIS 86331 (S.D.N.Y. May 12, 2022)   
3                 No. 13-cv-1215 (TSC), 2022 U.S. Dist. LEXIS 60922 (D.D.C. Mar. 31, 2022)   
4  Nos. 20-4097-cv(L), 20-4201-cv (XAP), 2022 U.S. App. LEXIS 8195 (2d Cir. Mar. 29, 2022)   

   year                                                               court  \
0  2022                United States Court of Appeals for the Ninth Circuit   
1  2022  United States District Court for the Southern District of New York   
2  2022  United States District Court for the Southern District of New York   
3  2022           United States District Court for the District of Columbia   
4  2022               United States Court of Appeals for the Second Circuit   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    key_facts  \
0                                                                                                                                                                                                                                                        Plaintiffs own the rights to a catalogue comprised of 16,000 photographs of Pablo Picasso’s work, which was originally compiled by Picasso’s friend in 1932 (the “Zervos Catalogue”). In 1995, after obtaining permission from Picasso’s estate to publish a work illustrating and describing works by Picasso, Defendants Alan Wofsy and his company Alan Wofsy & Associates began publishing The Picasso Project—–a series of volumes reproducing images of Picasso’s work, including 1,492 photographs from the Zervos Catalogue. Plaintiffs sued for copyright infringement. A French court held the photographs were protected by copyright because they “added creative features through deliberate choices of lighting, the lens, filters, [

In [6]:
fair_use_findings_df.describe(include='all').T

,count,unique,top,freq
title,251,248,"Bouchat v. Balt. Ravens Ltd. P’ship,",2
case_number,251,251,39 F.4th 1214 (9th Cir. 2022),1
year,251,56,2020,17
court,251,60,United States District Court for the Southern District of New York,48
key_facts,251,251,"Plaintiffs own the rights to a catalogue comprised of 16,000 photographs of Pablo Picasso’s work, which was originally compiled by Picasso’s friend in 1932 (the “Zervos Catalogue”). In 1995, after obtaining permission from Picasso’s estate to publish a work illustrating and describing works by Picasso, Defendants Alan Wofsy and his company Alan Wofsy & Associates began publishing The Picasso Project—–a series of volumes reproducing images of Picasso’s work, including 1,492 photographs from the Zervos Catalogue. Plaintiffs sued for copyright infringement. A French court held the photographs were protected by copyright because they “added creative features through deliberate choices of lighting, the lens, filters, [and] framing or angle of view.” In 2001, Plaintiffs obtained a judgment in France that subjected Defendants to damages for any further acts of infringement. In 2012, after discovering copies of The Picasso Project in a French bookstore, Plaintiffs enforced their judgment in France and were awarded €2 million. Plaintiffs sought recognition of the judgment in the U.S. courts. The district court granted summary judgment for Defendants, determining that the French judgment was “repugnant to U.S. public policy protecting free expression” because it failed to provide a fair use defense. Plaintiffs appealed; and Defendants cross-appealed on other defenses.",1
issue,251,248,Whether a university’s electronic distribution of unlicensed copyrighted works to students is a fair use.,2
holding,251,251,"The panel held that the first factor, the purpose and character of the use, weighed against fair use because Defendants conceded that The Picasso Project was a commercial venture and the use at issue—reproduction of the photographs in a book illustrating Picasso’s works—was not transformative. Specifically, the court noted that Defendants’ use “did not serve an ‘entirely different function’ than the originals,” but had overlapping purposes, and the insertion of informative captions did not “necessarily” transform the works. The second factor, the nature of the copyrighted work, did not favor fair use because, although the works were published and documentary in nature, the French court had concluded that the photographs exhibited creative elements. The court determined that the third factor, the amount and substantiality of the work used, weighed against fair use because Defendants failed to demonstrate that “copying the entirety of each photograph was necessary.” The fourth factor, the effect of the use upon the potential market for or value of the copyrighted work, also weighed against fair use because there is a presumption of market harm when the use is commercial and non-transformative. Although Defendants presented evidence that auction prices for the Zervos Catalogue increased while The Picasso Project was on the market, Defendants had not provided evidence that “widespread appropriation” of the works would not harm the market for the photographs. Weighing all the factors, the court had “serious doubts” that fair use would protect Defendants’ use, and, accordingly, granted summary judgment to Plaintiffs on the public policy defense.",1
tags,251,207,Second Circuit; Education/Scholarship/Research; Textual work,4
outcome,251,22,Fair use not found,99


### Conexion a Base de datos MYSQL

In [7]:
# Creamos objeto de conexion a base de datos
mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = "Ejercicio2024!"
)
 
print(mydb)

In [8]:
def create_database(cursor,DB_NAME):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(DB_NAME))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

In [9]:
mycursor = mydb.cursor()
create_database(mycursor, "Fair_use")

Failed creating database: 1007 (HY000): Can't create database 'Fair_use'; database exists


### Creacion de base de dato y tablas

In [10]:
mycursor.execute("USE Fair_use")

In [11]:
fair_use_cases_df.columns

Index(['case', 'year', 'court', 'jurisdiction', 'categories', 'outcome',
       'fair_use_found'],
      dtype='object')

In [12]:
#table = ("CREATE TABLE `titles` ("
#    "  `emp_no` int(11) NOT NULL,"
#    "  `title` varchar(50) NOT NULL,"
#    "  `from_date` date NOT NULL,"
#    "  `to_date` date DEFAULT NULL,"
#    "  PRIMARY KEY (`emp_no`,`title`,`from_date`), KEY `emp_no` (`emp_no`)"
#    ") ENGINE=InnoDB")

In [13]:
#mycursor.execute("DROP TABLE `fair_use_cases`")

In [14]:
mycursor.execute("CREATE TABLE `fair_use_cases` (`id` INT AUTO_INCREMENT PRIMARY KEY, `case` VARCHAR(255), `year` SMALLINT(255), `court` VARCHAR(255) ,`jurisdiction` VARCHAR(255), `categories` VARCHAR(255), `outcome` VARCHAR(255), `fair_use_found` VARCHAR(255))ENGINE=InnoDB")
#, year SMALLINT(255), jurisdiction VARCHAR(255), categories VARCHAR(255), outcome VARCHAR(255), fair_use_found VARCHAR(255)

### Consultas sql

In [15]:
hostname= "localhost"
database= "Fair_use"
username= "root"
password= "Ejercicio2024!"
engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}".format(host=hostname, db=database, user=username, pw=password))

In [16]:
fair_use_cases_df.to_sql('fair_use_cases', con=engine, if_exists='append', index=False)

251

In [17]:
mycursor.execute('SELECT * FROM fair_use_cases')
for row in mycursor.fetchall():
    print (row)

(1, 'De Fontbrune v. Wofsy, 39 F.4th 1214 (9th Cir. 2022)', 2022, '9th Circuit', '9th Circuit', 'Education/Scholarship/Research; Photograph', 'Fair use not found', '0')
(2, 'Sedlik v. Von Drachenberg, No. CV 21-1102 (C.D. Cal. May 31, 2022)', 2022, 'C.D. Cal.', '9th Circuit', 'Painting/Drawing/Graphic; Photograph', 'Preliminary finding; Fair use not found', '0')
(3, 'Sketchworks Indus. Strength Comedy, Inc. v. Jacobs, No. 19-CV-7470-LTS-VF (S.D.N.Y. May 12, 2022)', 2022, 'S.D.N.Y.', '2nd Circuit', 'Film/Audiovisual; Music; Parody/Satire; Review/Commentary', 'Fair use found', '1')
(4, "Am. Soc'y for Testing & Materials v. Public.Resource.Org, Inc., No. 13-cv-1215 (D.D.C. Mar. 31, 2022)", 2022, 'D.D.C.', 'District of Columbia Circuit', 'Education/Scholarship/Research; Textual Work; Used in government proceeding', 'Mixed Result', '0')
(5, 'Yang v. Mic Network Inc., Nos. 20-4097-cv, 20-4201-cv (2d Cir. Mar. 29, 2022)', 2022, '2d Circuit', '2nd Circuit', 'News reporting; Photography', 'Fair